In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import xarray as xr
import rioxarray as rxr
import codebase

In [13]:
datadir = '/global/scratch/users/ann_scheliga/aux_dam_datasets/GRDC_CRB/'
basin = 'niger'

In [3]:
res_shp = codebase.load_data.load_GRanD()

In [22]:
niger_basin_res = res_shp.loc[res_shp['MAIN_BASIN'] =='Niger']

In [14]:
subregions_meta = gpd.read_file(datadir + basin + '_subregions.geojson')
stations_meta = gpd.read_file(datadir + basin + '_stationbasins.geojson')

In [33]:
res_shp.columns

Index(['GRAND_ID', 'RES_NAME', 'DAM_NAME', 'ALT_NAME', 'RIVER', 'ALT_RIVER',
       'MAIN_BASIN', 'SUB_BASIN', 'NEAR_CITY', 'ALT_CITY', 'ADMIN_UNIT',
       'SEC_ADMIN', 'COUNTRY', 'SEC_CNTRY', 'YEAR', 'ALT_YEAR', 'REM_YEAR',
       'DAM_HGT_M', 'ALT_HGT_M', 'DAM_LEN_M', 'ALT_LEN_M', 'AREA_SKM',
       'AREA_POLY', 'AREA_REP', 'AREA_MAX', 'AREA_MIN', 'CAP_MCM', 'CAP_MAX',
       'CAP_REP', 'CAP_MIN', 'DEPTH_M', 'DIS_AVG_LS', 'DOR_PC', 'ELEV_MASL',
       'CATCH_SKM', 'CATCH_REP', 'DATA_INFO', 'USE_IRRI', 'USE_ELEC',
       'USE_SUPP', 'USE_FCON', 'USE_RECR', 'USE_NAVI', 'USE_FISH', 'USE_PCON',
       'USE_LIVE', 'USE_OTHR', 'MAIN_USE', 'LAKE_CTRL', 'MULTI_DAMS',
       'TIMELINE', 'COMMENTS', 'URL', 'QUALITY', 'EDITOR', 'LONG_DD', 'LAT_DD',
       'POLY_SRC', 'geometry'],
      dtype='object')

In [39]:
print(stations_meta.columns)
print(stations_meta.shape)

Index(['grdc_no', 'river', 'station', 'area', 'altitude', 'lat_org',
       'long_org', 'lat_pp', 'long_pp', 'dist_km', 'area_calc', 'quality',
       'type', 'comment', 'source', 'geometry'],
      dtype='object')
(110, 16)


In [42]:
stations_meta['lat_lon'] = gpd.points_from_xy(y =stations_meta['lat_org'], x =  stations_meta['long_org'])
stations_points = stations_meta.set_geometry('lat_lon', crs = "EPSG:4326")
stations_points.head(2)

,grdc_no,river,station,area,altitude,lat_org,long_org,lat_pp,long_pp,dist_km,area_calc,quality,type,comment,source,geometry,lat_lon
0,1134020.0,SANKARANI,GUELELINKORO,23300.0,-999.0,11.1500,-8.5500,11.15083,-8.55417,0.5,23585.0,High,Automatic,Area difference <= 5% and distance <= 5 km,merit,"POLYGON ((-8.96120 9.65790, -8.96210 9.65790, ...",POINT (-8.55000 11.15000)
1,1134030.0,RIVER NIGER,BANANKORO,71750.0,329.0,11.6833,-8.6667,11.68120,-8.66460,0.3,70767.7,High,Automatic,Area difference <= 5% and distance <= 5 km,hydrosheds,"POLYGON ((-10.20000 11.91670, -10.19170 11.916...",POINT (-8.66670 11.68330)


In [1]:
## Code runs, commented for file size
# m = stations_meta.explore(style_kwds={'color':'green','fill':False})
# niger_basin_res.explore(m=m,style_kwds=dict(color='blue'),tooltip=['DAM_NAME','RES_NAME','RIVER','AREA_SKM'])
# stations_points.explore(m=m,style_kwds=dict(color='red'),tooltip=['grdc_no','station','river','quality'])
# m